In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2022-ml-finalexam-p5/train.csv
/kaggle/input/2022-ml-finalexam-p5/test.csv
/kaggle/input/2022-ml-finalexam-p5/sample_submit.csv


train.csv / test.csv : 타이타닉 탑승자들 중 일부의 인적 정보와 생존 여부 데이터
PassengerID : 탑승객 고유 아이디
Survival : 탑승객 생존 유무 (0: 사망, 1: 생존)
Pclass : 등실의 등급
Name : 이름
Sex : 성별
Age : 나이
Sibsp : 함께 탐승한 형제자매, 아내, 남편의 수
Parch : 함께 탐승한 부모, 자식의 수
Ticket :티켓 번호
Fare : 티켓의 요금
Cabin : 객실번호
Embarked : 배에 탑승한 항구 이름 ( C = Cherbourn, Q = Queenstown, S = Southampton)

문제 설명 : 타이타닉 생존자 예측 문제
1912년 4월 15일, 타이타닉은 항해중 빙산과의 충돌로 인해 침몰합니다. 탑승인원 2224명 중 1502명이 사망한 비극적인 사건이었고, 선박의 안전규정을 개선시키는 계기가 되었습니다.
여러분은 기계학습을 이용한 어떤 종류의 사람들이 많이 살아남았는지 예측해 봅시다.

In [2]:
train = pd.read_csv('/kaggle/input/2022-ml-finalexam-p5/train.csv')
test = pd.read_csv('/kaggle/input/2022-ml-finalexam-p5/test.csv')
#데이터를 불러온다

In [3]:
train
#각각의 데이터 파악

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
595,596,0,3,"Van Impe, Mr. Jean Baptiste",male,36.0,1,1,345773,24.1500,NaN,S
596,597,1,2,"Leitch, Miss. Jessie Wills",female,NaN,0,0,248727,33.0000,NaN,S
597,598,0,3,"Johnson, Mr. Alfred",male,49.0,0,0,LINE,0.0000,NaN,S
598,599,0,3,"Boulos, Mr. Hanna",male,NaN,0,0,2664,7.2250,NaN,C


In [4]:
test
#각각의 데이터 파악

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,601,2,"Jacobsohn, Mrs. Sidney Samuel (Amy Frances Chr...",female,24.0,2,1,243847,27.0000,NaN,S
1,602,3,"Slabenoff, Mr. Petco",male,NaN,0,0,349214,7.8958,NaN,S
2,603,1,"Harrington, Mr. Charles H",male,NaN,0,0,113796,42.4000,NaN,S
3,604,3,"Torber, Mr. Ernst William",male,44.0,0,0,364511,8.0500,NaN,S
4,605,1,"Homer, Mr. Harry (""Mr E Haven"")",male,35.0,0,0,111426,26.5500,NaN,C
...,...,...,...,...,...,...,...,...,...,...,...
286,887,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
287,888,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
288,889,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
289,890,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [5]:
train = train.drop(['Name','Ticket','Cabin','Age'],axis =1)
test = test.drop(['Name','Ticket','Cabin','Age'],axis =1)
#필요없는 데이터들을 드랍

In [6]:
train_x = train.drop(['Survived'],axis =1)
test_x = test
train_y = train['Survived']
#train_x, test_x, train_y 구분짓기

In [7]:
train_x#정리된 train_x 확인

,PassengerId,Pclass,Sex,SibSp,Parch,Fare,Embarked
0,1,3,male,1,0,7.2500,S
1,2,1,female,1,0,71.2833,C
2,3,3,female,0,0,7.9250,S
3,4,1,female,1,0,53.1000,S
4,5,3,male,0,0,8.0500,S
...,...,...,...,...,...,...,...
595,596,3,male,1,1,24.1500,S
596,597,2,female,0,0,33.0000,S
597,598,3,male,0,0,0.0000,S
598,599,3,male,0,0,7.2250,C


In [8]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(train_x['Sex'])
train_x['Sex'] = le.transform(train_x['Sex'])
test_x['Sex'] = le.transform(test_x['Sex'])

le = LabelEncoder()
le.fit(train_x['Embarked'])
train_x['Embarked'] = le.transform(train_x['Embarked'])
test_x['Embarked'] = le.transform(test_x['Embarked'])
#labelEncoder로 필요한 것들을 숫자로 변경한다.

In [9]:
from sklearn.ensemble import RandomForestClassifier
knn = RandomForestClassifier()
knn.fit(train_x,train_y)
y_pred = knn.predict(test_x)
#0 1로 구분하기 때문에 분류를 쓴다. 그러나 svc는 필요가 없고 knn이나 logisitic도 낮은점수가 나오고 데이터의 형태가 이를 필요한 형태가 아니다
#따라서 남은 RandomForestClassifier를 사용하였다.

In [10]:
submit = pd.read_csv('/kaggle/input/2022-ml-finalexam-p5/sample_submit.csv')
#submit 불러오기

In [11]:
submit['Survived']= y_pred
submit = submit.to_csv('submit.csv',index=False)
#저장해준다.